In [1]:
# BASIC IMPORTS OF ALL LIBRARIES
'''
Glove Imports are 300-dim from GENSIM
Pickle saves all the datasets
'''
from nltk import pos_tag
import csv
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx as nx
from copy import deepcopy
import pickle
from entity_groups import *
from graphviz import Source
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 
import gensim.downloader as api

wnl = WordNetLemmatizer()
word_vectors = api.load("glove-wiki-gigaword-300")

In [2]:
# junk verbs that are generalizably illegal
junk_rel = ["be","have","do","let","","make","tell","say","describe","decide","want","name","know","think","try","become","oneday","put","come",'see', 'need', 'look', 'help', 'come', 'take', 'get', 'put', 'pick', 'turn', 'go', 'stand', 'give', 'notice', 'use',"get","start"]

# hypothetical words that are generally useless / qualifiers
junk_words = ["would","could","should","maybe","perhaps","think","might","assume","claim","this"]

# negative sentences
junk_words.extend(['n\'t',"not"])
print(junk_words)

# perspective actants that are pointless
junk = ["you","i","we","the","it","he","she","steinbeck","people","author",
        "book","me","steinback","him","her","their","this","","shelley","mary","harper","lee","tolkien"]

['would', 'could', 'should', 'maybe', 'perhaps', 'think', 'might', 'assume', 'claim', 'this', "n't", 'not']


In [3]:
########################## EDIT DATE ##########################

DATE = "0412"
file_name = "OS_News/"
text = "OS_News"
#text = "the_hobbit"

{'us': ['us'], 'you': ['you'], 'me': ['me'], 'flu': ['flu', 'fever', 'virus'], 'we': ['we'], 'they': ['they'], 'them': ['them'], '5g': ['5g'], 'china': ['china', 'wuhan'], 'scam': ['scam'], 'dog': ['dogs', 'doggos', 'dog', 'doggo'], 'radiation': ['radiation'], 'waves': ['wave', 'waves'], 'fact': ['fact'], 'towers': ['towers', 'tower', 'radiotowers'], 'risk': ['risk'], 'cat': ['cat', 'cats'], 'bat': ['bat', 'bats']}
{'us': 'us', 'you': 'you', 'me': 'me', 'flu': 'flu', 'fever': 'flu', 'virus': 'flu', 'we': 'we', 'they': 'they', 'them': 'them', '5g': '5g', 'china': 'china', 'wuhan': 'china', 'scam': 'scam', 'dogs': 'dog', 'doggos': 'dog', 'dog': 'dog', 'doggo': 'dog', 'radiation': 'radiation', 'wave': 'waves', 'waves': 'waves', 'fact': 'fact', 'towers': 'towers', 'tower': 'towers', 'radiotowers': 'towers', 'risk': 'risk', 'cat': 'cat', 'cats': 'cat', 'bat': 'bat', 'bats': 'bat'}


In [9]:
results_path = "Results" + DATE + "/" + text
adj_mat_path = "Precedence_Matrix/" + text
abs_adj_mat_path = "AbsPrecedence_Matrix/" + text
dic_path = "Dictionary/" + text + ".pickle"
ner_file = "../Covid_19_Data/Extracted_Data/OS_News/" + "into_relex_final_relations_-1.csv"
entity_file = "../Covid_19_Data/Extracted_Data/OS_News/" + "df_ent_final_ranking.csv"
S_use = {}
# fill this in

fileName = ner_file

In [10]:
# HELPER FUNCTION
def hypothetical_sentence(sent):
    '''
    Removes sentences that have the hypothetical list of words in them. 
    Does not offer information about sequencing.
    '''
    for word in sent.split():
        for ground_word in junk_words:
            if ground_word in word:
                return True
    return False

In [11]:
trajectories = []
same_post = 0
traj = ["START"]

with open(fileName) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        FLAG = 0

        sub_r = ""
        obj_r = ""
        sub_verb = ""

        if hypothetical_sentence(row[2]):
            continue

        try:
            sub = re.search(r"\{(\w+)\}", row[3]).group(1).lower()
            rel = wnl.lemmatize(re.search(r"\{(\w+)\}", row[4]).group(1).lower(), pos='v')
            obj = re.search(r"\{(\w+)\}", row[5]).group(1).lower()
        except:
            continue

        words = row[2].split()
        rel_words = []

        if rel == None or obj == None or sub == None:
            continue
        
        if same_post == row[0]:
            pass
        else:
            same_post = row[0]
            traj.append("TERMINATE")
            trajectories.append(traj)
            traj = ["START"]

        try:
            sub_r = S_use[sub]
        except:
            sub_r = sub
            FLAG += 1
        
        try:
            obj_r = S_use[obj]
        except:
            obj_r = obj
            FLAG += 1

        sub_verb = ""

        try:
            sub_verb = C_use[rel]
        except:
            sub_verb = rel

        if sub_r == obj_r or sub_r in junk or obj_r in junk or sub_verb in junk_rel or FLAG == 2:
            continue

        traj.append(sub_verb)
        traj.append(sub_r + "_" + obj_r)

# if we missed out on last post
if traj != "START":
    traj.append("TERMINATE")
    trajectories.append(traj)

print(trajectories)

m', 'lose', 'us_lives', 'report', 'they_sunday', 'report', 'they_midday', 'report', 'they_cases', 'complain', 'workers_they', 'find', 'they_cure', 'report', 'china_all', 'report', 'china_sunday', 'report', 'china_cases', 'report', 'china_sunday', 'report', 'china_cases', 'remain', 'they_april', 'remain', 'they_effect', 'confirm', 'china_cases', 'confirm', 'china_week', 'confirm', 'china_cases', 'lead', 'china_spread', 'selfquarantine', 'they_that', 'criticize', 'trump_us', 'consider', 'trump_us', 'state', 'us_deaths', 'add', 'us_cases', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'test', 'anaesthetist_flu', 'test', 'anaesthetist_flu', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'distance', 'china_flu', 'eat', 'ones_bat', 'eat', 'ones_bat', 'consume', 'bat_purposes', 'consume', 'bat_purposes', 'assure', 'wisdom_they', 'TERMINATE'], ['START', 'transmit', 'members_flu', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['